In [11]:
import pandas as pd
import sqlite3
import pyodbc
from sqlalchemy import create_engine
import urllib

In [12]:
#NorthWind database
conn_str_nw = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=MAX-PC-DEC-2023\SQLEXPRESS;'
    r'DATABASE=northwind;'
    r'Trusted_Connection=yes;'
)

# URL-encode the connection string
quoted_conn_str_nw = urllib.parse.quote_plus(conn_str_nw)

# Create the SQLAlchemy engine using the URL-encoded connection string
engine_nw = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str_nw}")

In [13]:
#Adventureworks database
conn_str_aw = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=MAX-PC-DEC-2023\SQLEXPRESS;'
    r'DATABASE=Adventureworks2019;'
    r'Trusted_Connection=yes;'
)

# URL-encode the connection string
quoted_conn_str_aw = urllib.parse.quote_plus(conn_str_nw)

# Create the SQLAlchemy engine using the URL-encoded connection string
engine_aw = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str_aw}")

In [14]:
engine_ae = create_engine('access+pyodbc:///?odbc_connect=' +
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\\Users\\Maxhb\Documents\\GitHub\\UnitedOutdoors\\data\\aenc.accdb;')

In [15]:
# SQL queries with updated aliases
northwind_query = """
SELECT EmployeeID AS Id, 'NW' AS [Database], FirstName, LastName, Address, City, Region, PostalCode, Country, HomePhone AS PhoneNumber, NULL AS Sex, HireDate AS StartDate, ReportsTo AS ManagerId, NULL AS TerminationDate
FROM Employees
"""

adventure_query = """
SELECT BusinessEntityID AS Id, 'AW' AS [Database], FirstName, LastName, AddressLine1 AS Address, City, StateProvinceName AS Region, PostalCode, CountryRegionCode AS Country, PhoneNumber, Gender AS Sex, HireDate AS StartDate, NULL AS ManagerId, NULL AS TerminationDate
FROM HumanResources.Employee AS e
JOIN Person.Person AS p ON e.BusinessEntityID = p.BusinessEntityID
JOIN Person.BusinessEntityAddress AS bea ON p.BusinessEntityID = bea.BusinessEntityID
JOIN Person.Address AS a ON bea.AddressID = a.AddressID
JOIN Person.StateProvince AS sp ON a.StateProvinceID = sp.StateProvinceID
JOIN Person.PersonPhone AS pp ON p.BusinessEntityID = pp.BusinessEntityID
"""

aenc_query = """
SELECT EmployeeID AS Id, 'AENC' AS [Database], FirstName, LastName, Address, City, StateName AS Region, PostalCode, Country, PhoneNumber, Sex, HireDate AS StartDate, ManagerID, TerminationDate
FROM Employee
JOIN State ON Employee.StateID = State.StateID
"""

# Function to execute query and return data frame
def fetch_data(query, engine):
    try:
        return pd.read_sql_query(query, engine)
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# Fetch data from databases
northwind_data = fetch_data(northwind_query, conn_str_nw)
adventure_data = fetch_data(adventure_query, conn_str_aw)
aenc_data = fetch_data(aenc_query, engine_ae)

# Combine all data into a single DataFrame
combined_data = pd.concat([northwind_data, adventure_data, aenc_data], ignore_index=True)

# Display the combined data
print(combined_data.head())

# Optional: Save the combined data to a new SQL table or a CSV file
# combined_data.to_csv('combined_employees.csv', index=False)

An error occurred: Could not parse SQLAlchemy URL from string 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=MAX-PC-DEC-2023\SQLEXPRESS;DATABASE=northwind;Trusted_Connection=yes;'
An error occurred: Could not parse SQLAlchemy URL from string 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=MAX-PC-DEC-2023\SQLEXPRESS;DATABASE=Adventureworks2019;Trusted_Connection=yes;'
An error occurred: (pyodbc.Error) ('HY000', '[HY000] [Microsoft][ODBC Microsoft Access-stuurprogramma] Kan het bestand (onbekend) niet vinden. (-1811) (SQLDriverConnect); [HY000] [Microsoft][ODBC Microsoft Access-stuurprogramma] Kan het bestand (onbekend) niet vinden. (-1811)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)
Empty DataFrame
Columns: []
Index: []
